# Functions and Composition

**Julia** wants, as far as possible, to allow you to write expressions in a manner natural to mathematicians. 

We've seen that it naturally supports both Unicode (not unusual for a programming language) and LaTeX-style shortcuts for representing mathematical symbols; however, Julia also extends its philosophy to different ways of representing composition of functions.

To demonstrate this, we'll start with a classic "long-form" function, complete with doc-string:

In [1]:
"multiply(a,b): return the product of a and b"
function multiply(a, b)
    a*b
end

multiply(2,3)

6

In most traditional imperative languages, you "compose" functions by simply "nesting" the function calls.
Julia supports this, here composing two trivial anonymous functions:

In [2]:
twice = x -> 2*x
thrice = x -> 3*x

twice(thrice(5))

30

However, Julia also supports a more "mathematically natural" method of function composition - using the ∘ (\circ) operator, we can write a very natural composition operation, and then simply "call the resulting function":

In [3]:
six_times = twice ∘ thrice

six_times(5)

30

In [4]:
(twice ∘ thrice)(5) 

30

In [5]:
(twice ∘ multiply)(2,3)

12

(Clearly, this does not work the other way around in the latter case, as _multiply_ needs two arguments...)

Function composition is especially nice if you use it with broadcasting (see next...), or with functional programming operators like map(). 
Because the composition operator is a fundamental component of Julia, "composed functions" will also be JITed separately to their component functions - and thus can be more efficient than simply naïvely nesting function calls.
We can demonstrate this using Julia's code introspection features:

the macro "@code_llvm" shows us the resulting code that will be compiled by the JIT for the given expression - in this case as LLVM Intermediate Representation (there are other @code_* macros that show the code at other levels of processing)

In [6]:
@code_llvm( (twice ∘ thrice)(2) )

;  @ operators.jl:1031 within `ComposedFunction`
; Function Attrs: uwtable
define i64 @julia_ComposedFunction_972(i64 signext %0) #0 {
top:
; ┌ @ operators.jl:1031 within `#_#97`
; │┌ @ operators.jl:1034 within `call_composed`
; ││┌ @ In[2]:1 within `#3`
; │││┌ @ int.jl:88 within `*`
      %1 = mul i64 %0, 6
; └└└└
  ret i64 %1
}


In [7]:
@code_llvm( twice(thrice(2)))

;  @ In[2]:1 within `#3`
; Function Attrs: uwtable
define i64 @"julia_#3_997"(i64 signext %0) #0 {
top:
; ┌ @ int.jl:88 within `*`
   %1 = shl i64 %0, 1
; └
  ret i64 %1
}


As should be clear, the composed form is directly optimised to "multiply by 6", whilst the nested function call simply nests the function (@code_llvm only shows us the outer function - "twice", which uses a "shift left" to efficiently multiply by 2 ).